References:
1. Train/validation/test splits in sklearn: https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn

2. Assumptions for logistic regression:
https://pubmed.ncbi.nlm.nih.gov/21996075/ & 
https://www.statology.org/assumptions-of-logistic-regression/

3. Weighted logistic regression: https://www.statsmodels.org/dev/examples/notebooks/generated/glm_weights.html

4. Backward stepwise selection based on p-value: https://towardsdatascience.com/backward-elimination-for-feature-selection-in-machine-learning-c6a3a8f8cef4

In [575]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
import duckdb
from statsmodels.api import Logit
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [576]:
main_df = pd.read_csv("dating_main.csv")

In [577]:
print(main_df.shape)
main_df.head()

(5585, 25)


,Unnamed: 0,id,gender,wave,num_dates,partner_id,age,field_cd,race,career_c,...,decision,rate_p_attr,rate_p_sinc,rate_p_intel,rate_p_fun,rate_p_amb,field,race_cd,career,gender_cd
0,30,4,Female,1,10,11,23,1,European/Caucasian-American,1,...,0,4,10,8,5,8,Law,2,Lawyer,0
1,31,4,Female,1,10,12,23,1,European/Caucasian-American,1,...,0,8,7,8,10,7,Law,2,Lawyer,0
2,32,4,Female,1,10,13,23,1,European/Caucasian-American,1,...,0,4,7,8,8,6,Law,2,Lawyer,0
3,33,4,Female,1,10,14,23,1,European/Caucasian-American,1,...,1,8,10,7,10,7,Law,2,Lawyer,0
4,34,4,Female,1,10,15,23,1,European/Caucasian-American,1,...,0,6,9,8,9,8,Law,2,Lawyer,0


## Main_df

1. Split dating_df into training and test sets
2. Check for assumptions for logit on the training set.
3. Fit a logistic regression for dating_df using statsmodel (with 1/num_dates as weights!!!).
4. Interpret coefficients & discuss statistical significance. Comment on gender difference
5. Perform model selection: pick M0, …, Mp based on training error rate
6. Then do cross-validation (because maybe it performs well on the training set but not the test set)


### 1. Create interaction terms

In [578]:
# gender_cd 1 = Male; 0 = Female
main_df['gender_p_attr'] = main_df["gender_cd"] * main_df["rate_p_attr"] # gender_cd*rate_p_attr
main_df['gender_p_sinc'] = main_df["gender_cd"] * main_df["rate_p_sinc"] # gender_cd*rate_p_sinc
main_df['gender_p_intel'] = main_df["gender_cd"] * main_df["rate_p_intel"] # gender_cd*rate_p_intel
main_df['gender_p_fun'] = main_df["gender_cd"] * main_df["rate_p_fun"] # gender_cd*rate_p_fun
main_df['gender_p_amb'] = main_df["gender_cd"] * main_df["rate_p_amb"] # gender_cd*rate_p_amb

main_X_vars = ['rate_p_attr', 'rate_p_intel', 'rate_p_fun', 'rate_p_amb', 'rate_p_sinc', 
              'gender_p_attr', 'gender_p_intel', 'gender_p_fun', 'gender_p_amb', 'gender_p_sinc']

### 2. Split main_df into training/validation/test sets

In [579]:
# 70% to training set; 15% to validation; 15% to test
SEED = 7777
main_train, main_val_and_test = train_test_split(main_df, test_size=.3, random_state = SEED)
main_val, main_test = train_test_split(main_val_and_test, test_size=.5, random_state = SEED)
# main_test = main_test.drop_duplicates(subset=['id'])
# main_val = main_val.drop_duplicates(subset=['id'])

# train: recalculate weight
# test & val: drop duplicates

# Reference: see source 1 in References

In [580]:
print(main_train.shape)
print(main_val.shape)
print(main_test.shape)

(3909, 30)
(838, 30)
(838, 30)


From now on, focus on the training set.

### 3. Check assumptions for multivariate logistic regression on the training set

Reference: see source 2 in References

#### (a) Independent observations

"Logistic regression assumes that the observations in the dataset are independent of each other. That is, the observations should not come from repeated measurements of the same individual or be related to each other in any way."

One common way to check this assumption is to make a plot of residual vs the order of observations and investigate any non-random patterns. At this step we have not fitted the model, so we cannot make the plot yet.

However, we know for sure the observations are NOT independent because of how the data was created. For example, say the first row represents the date between participant 1 and participant 2 and the second row represents the date between participant 1 and participant 3. Then the first two observations are dependent because the ratings, which are our explanatory variables, and the decision (our response variable) are given by the same person. They can take different values, but it is quite intuitive the same participant's ratings and decision towards different dates are dependent because of their unique rating/decision system and other personal bias.

Notice that some participants dated more people than others in the experiment. To ensure participants making more decisions are not over-weighted in the model (and therefore address the abovementioned problem), the original paper weighted each observation by the inverse of num_dates (number of people the participant met in the event).

#### (b) Absence of multicollinearity

"Multicollinearity occurs when two or more explanatory variables are highly correlated to each other, such that they do not provide unique or independent information in the regression model."

We now check this assumption with correlation coefficients.

In [581]:
main_train_X = main_train[['rate_p_attr', 'rate_p_intel', 'rate_p_fun', 'rate_p_amb', 'rate_p_sinc']]
main_train_X.corr()

,rate_p_attr,rate_p_intel,rate_p_fun,rate_p_amb,rate_p_sinc
rate_p_attr,1.000000,0.378713,0.592064,0.354465,0.401095
rate_p_intel,0.378713,1.000000,0.496378,0.616104,0.638001
rate_p_fun,0.592064,0.496378,1.000000,0.496977,0.501193
rate_p_amb,0.354465,0.616104,0.496977,1.000000,0.437044
rate_p_sinc,0.401095,0.638001,0.501193,0.437044,1.000000


There is no pair of explanatory variables with high correlation coefficient (e.g., > 0.7), so it seems the assumption is not violated.

#### (c) Lack of strongly influential outliers

All the rating variables range from 1 to 10, and from their distribution graphs in EDA, we observe no outliers. All the interaction terms are binary, so there are no outliers.

#### (d) Binary response

In [582]:
main_df["decision"].unique()

array([0, 1])

Yes, it is binary.

### 3. Fit a weighted logistic regression.

Reference: source 3

In [583]:
# re-calculate num_dates and create weights
main_train["weight"] = 0

temp = []
for _, row in main_train.iterrows():
    row['num_dates'] = sum(main_train['id'] == row['id'])

main_train["weight"] = 1/main_train["num_dates"]

# make the regression formula
formula = "decision ~ "
for var in main_X_vars:
    formula = formula + var + "+"
formula = formula[:-1]

# fit the model
glm = smf.glm(formula,
              data=main_train, 
              family=sm.families.Binomial(),
              freq_weights=main_train['weight'])
model = glm.fit()
print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               decision   No. Observations:                 3909
Model:                            GLM   Df Residuals:                   245.12
Model Family:                Binomial   Df Model:                           10
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -134.94
Date:                Thu, 16 Nov 2023   Deviance:                       269.87
Time:                        21:47:43   Pearson chi2:                     269.
No. Iterations:                     5   Pseudo R-squ. (CS):            0.02066
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -5.1375      0.998     -5.

In [584]:
# training error rate
round(np.mean((model.predict(main_train[main_X_vars]) > 0.5) != main_train["decision"]), 3)

0.262

### 4. Interpret coefficients & discuss significance

### 5. Backward stepwise selection based on p-value

Reference: see source 4

The explanatory variable with the highest p-value in the previous model was rate_p_sinc (0.913). Let's drop this variable and refit the model.

In [585]:
formula = "decision ~ "
for var in main_X_vars:
    if var != "rate_p_sinc":
        formula = formula + var + "+"
formula = formula[:-1]

glm = smf.glm(
    formula,
    data=main_train,
    family=sm.families.Binomial(),
    freq_weights=main_train['weight']
)
model9 = glm.fit()
print(model9.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               decision   No. Observations:                 3909
Model:                            GLM   Df Residuals:                   246.12
Model Family:                Binomial   Df Model:                            9
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -134.96
Date:                Thu, 16 Nov 2023   Deviance:                       269.91
Time:                        21:47:43   Pearson chi2:                     269.
No. Iterations:                     5   Pseudo R-squ. (CS):            0.02065
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -5.1562      0.994     -5.

In [586]:
round(np.mean((model9.predict(main_train[main_X_vars]) > 0.5) != main_train["decision"]), 3)

0.261

Then drop gender_p_amb (p-value = 0.827).

In [587]:
formula = "decision ~ "
for var in main_X_vars:
    if var not in ["rate_p_sinc", "gender_p_amb"]:
        formula = formula + var + "+"
formula = formula[:-1]

glm = smf.glm(
    formula,
    data=main_train,
    family=sm.families.Binomial(),
    freq_weights=main_train['weight']
)
model8 = glm.fit()
print(model8.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               decision   No. Observations:                 3909
Model:                            GLM   Df Residuals:                   247.12
Model Family:                Binomial   Df Model:                            8
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -134.96
Date:                Thu, 16 Nov 2023   Deviance:                       269.91
Time:                        21:47:44   Pearson chi2:                     269.
No. Iterations:                     5   Pseudo R-squ. (CS):            0.02065
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -5.1562      0.994     -5.

In [588]:
round(np.mean((model8.predict(main_train[main_X_vars]) > 0.5) != main_train["decision"]), 3)

0.261

Then drop gender_p_fun.

In [589]:
formula = "decision ~ "
for var in main_X_vars:
    if var not in ["rate_p_sinc", "gender_p_amb", "gender_p_fun"]:
        formula = formula + var + "+"
formula = formula[:-1]

glm = smf.glm(
    formula,
    data=main_train,
    family=sm.families.Binomial(),
    freq_weights=main_train['weight'],
)
model7 = glm.fit()
print(model7.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               decision   No. Observations:                 3909
Model:                            GLM   Df Residuals:                   248.12
Model Family:                Binomial   Df Model:                            7
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -135.02
Date:                Thu, 16 Nov 2023   Deviance:                       270.04
Time:                        21:47:44   Pearson chi2:                     268.
No. Iterations:                     5   Pseudo R-squ. (CS):            0.02061
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -5.1650      0.994     -5.

In [590]:
round(np.mean((model7.predict(main_train[main_X_vars]) > 0.5) != main_train["decision"]), 3)

0.262

Then drop gender_p_intel.

In [591]:
formula = "decision ~ "
for var in main_X_vars:
    if var not in ["rate_p_sinc", "gender_p_amb", "gender_p_fun", "gender_p_intel"]:
        formula = formula + var + "+"
formula = formula[:-1]

glm = smf.glm(
    formula,
    data=main_train,
    family=sm.families.Binomial(),
    freq_weights=main_train['weight'],
)
model6 = glm.fit()
print(model6.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               decision   No. Observations:                 3909
Model:                            GLM   Df Residuals:                   249.12
Model Family:                Binomial   Df Model:                            6
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -135.18
Date:                Thu, 16 Nov 2023   Deviance:                       270.36
Time:                        21:47:44   Pearson chi2:                     270.
No. Iterations:                     5   Pseudo R-squ. (CS):            0.02053
Covariance Type:            nonrobust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -5.0860      0.980     -5.191

In [592]:
round(np.mean((model6.predict(main_train[main_X_vars]) > 0.5) != main_train["decision"]), 3)

0.262

Then drop rate_p_intel.

In [593]:
formula = "decision ~ "
for var in main_X_vars:
    if var not in ["rate_p_sinc", "gender_p_amb", "gender_p_fun", "gender_p_intel", "rate_p_intel"]:
        formula = formula + var + "+"
formula = formula[:-1]

glm = smf.glm(
    formula,
    data=main_train,
    family=sm.families.Binomial(),
    freq_weights=main_train['weight'],
)
model5 = glm.fit()
print(model5.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               decision   No. Observations:                 3909
Model:                            GLM   Df Residuals:                   250.12
Model Family:                Binomial   Df Model:                            5
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -135.35
Date:                Thu, 16 Nov 2023   Deviance:                       270.70
Time:                        21:47:44   Pearson chi2:                     273.
No. Iterations:                     5   Pseudo R-squ. (CS):            0.02045
Covariance Type:            nonrobust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -4.8400      0.877     -5.517

In [594]:
round(np.mean((model5.predict(main_train[main_X_vars]) > 0.5) != main_train["decision"]), 3)

0.261

In [595]:
formula = "decision ~ "
for var in main_X_vars:
    if var not in ["rate_p_sinc", "gender_p_amb", "gender_p_fun", "gender_p_intel", "rate_p_intel", "rate_p_amb"]:
        formula = formula + var + "+"
formula = formula[:-1]

glm = smf.glm(
    formula,
    data=main_train,
    family=sm.families.Binomial(),
    freq_weights=main_train['weight'],
)
model4 = glm.fit()
print(model4.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               decision   No. Observations:                 3909
Model:                            GLM   Df Residuals:                   251.12
Model Family:                Binomial   Df Model:                            4
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -135.78
Date:                Thu, 16 Nov 2023   Deviance:                       271.57
Time:                        21:47:44   Pearson chi2:                     270.
No. Iterations:                     5   Pseudo R-squ. (CS):            0.02023
Covariance Type:            nonrobust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -5.1575      0.817     -6.316

In [596]:
round(np.mean((model4.predict(main_train[main_X_vars]) > 0.5) != main_train["decision"]), 3)

0.265

In [597]:
formula = "decision ~ "
for var in main_X_vars:
    if var not in ["rate_p_sinc", "gender_p_amb", "gender_p_fun", "gender_p_intel", 
                   "rate_p_intel", "rate_p_amb", "gender_p_sinc"]:
        formula = formula + var + "+"
formula = formula[:-1]

glm = smf.glm(
    formula,
    data=main_train,
    family=sm.families.Binomial(),
    freq_weights=main_train['weight'],
)
model3 = glm.fit()
print(model3.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               decision   No. Observations:                 3909
Model:                            GLM   Df Residuals:                   252.12
Model Family:                Binomial   Df Model:                            3
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -136.83
Date:                Thu, 16 Nov 2023   Deviance:                       273.67
Time:                        21:47:44   Pearson chi2:                     275.
No. Iterations:                     5   Pseudo R-squ. (CS):            0.01970
Covariance Type:            nonrobust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -5.4849      0.795     -6.898

Now all the p-values < 0.1. So we stop. We are surprised the training error rates. Typically as the number of predictors decreases, the training error rate increases. But when we reduced the 10-predictor model all the way to this 3-predictor model, the training error rates fluctuated between about 0.255 to 0.26. Because the training error rates are really similar, there is no model that is noticeably better than the others in term of prediction power. Now we can use the validation set to test how these models perform on new data.

6. Validation set

In [598]:
main_val_X = main_val[main_X_vars]

In [599]:
round(np.mean((model3.predict(main_val_X) > 0.5) != main_val["decision"]), 3)

0.243

In [600]:
round(np.mean((model4.predict(main_val_X) > 0.5) != main_val["decision"]), 3)

0.247

In [601]:
round(np.mean((model5.predict(main_val_X) > 0.5) != main_val["decision"]), 3)

0.243

In [602]:
round(np.mean((model6.predict(main_val_X) > 0.5) != main_val["decision"]), 3)

0.243

In [603]:
round(np.mean((model7.predict(main_val_X) > 0.5) != main_val["decision"]), 3)

0.241

In [604]:
round(np.mean((model8.predict(main_val_X) > 0.5) != main_val["decision"]), 3)

0.24

In [605]:
round(np.mean((model9.predict(main_val_X) > 0.5) != main_val["decision"]), 3)

0.24

In [606]:
round(np.mean((model.predict(main_val_X) > 0.5) != main_val["decision"]), 3)

0.239

The 3-predictor model did the best in fitting the validation set (smallest validation error). So we would choose it as our final model. Now let's test it one last time with the test set.

In [607]:
main_test_X = main_test[main_X_vars]
round(np.mean((model3.predict(main_test_X) > 0.5) != main_test["decision"]), 3)

0.239

The 3-predictor model did even better on the test set than the training and validation sets.